# LangChainを使用したAmazon Bedrockによる検索拡張質問と回答

> *このノートブックは SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

### コンテキスト
前回、モデルがタイヤの交換方法を回答することを確認しましたが、我々は関連データを手動で提供し、コンテキストを自分たちで提供する必要がありました。私たちは、Bedrockで利用可能なモデルを活用し、トレーニング中に学習した知識に基づいて質問し、手動でコンテキストを提供するアプローチを探りました。このアプローチは短いドキュメントやシングルトンのアプリケーションでは機能しますが、モデルに送信されるプロンプトにすべてが収まらない大規模なエンタープライズドキュメントが存在する可能性があるエンタープライズレベルの質問応答には拡張することができません。

### パターン
Retreival Augmented Generation (RAG) と呼ばれるアーキテクチャを実装することで、このプロセスを改善することができます。RAG は言語モデルの外部でデータを検索し（ノンパラメトリック）、関連する検索データをコンテキストに追加することでプロンプトを補強します。

このノートブックでは、ユーザーの質問に対する答えを提供するために文書を検索し、活用する質問応答のパターンへのアプローチ方法を説明します。

### チャレンジ
- トークン制限を超える大きな文書を管理する方法
- 質問に関連する文書を見つける方法

### 提案
上記の課題に対して、このノートブックは以下の戦略を提案します。

#### ドキュメントの準備
![Embeddings](./images/Embeddings_lang.png)

質問に答える前に、ドキュメントを処理し、ドキュメントストアインデックスに格納する必要があります。
- ドキュメントを読み込む
- 処理して小さなチャンクに分割する
- Amazon Bedrock Titan Embeddings モデルを使用して、各チャンクの数値ベクトル表現を作成する
- チャンクと対応する埋め込みを使用してインデックスを作成する

### 質問をする
![Question](./images/Chatbot_lang.png)

文書インデックスが準備できたら、質問を行い、質問に基づいて関連文書を検索することができるようになります。以下のステップが実行されます。
- 入力された質問の埋め込みを作成する
- 質問の Embedding をインデックスの埋め込みと比較する
- (上位 N 件の) 関連するドキュメントのチャンクを取得する
- これらのチャンクをプロンプトのコンテキストの一部として追加する
- Amazon Bedrock のモデルにプロンプトを送信する
- 取得した文書に基づいて、文脈に沿った回答を得る

## ユースケース
#### データセット
このアーキテクチャパターンを説明するために、IRS のドキュメントを使用します。これらの文書では、以下のようなトピックが説明されています：
- オリジネーション時の元本割引（OID）操作
- IRS への 10,000 ドルを超える現金支払いの報告
- 雇用者向けの税務ガイド

#### ペルソナ
IRS の仕組みや、ある行動がどのような意味を持つのか、また持たないのかを理解していない素人を想定してみましょう。
モデルは文書に基づいて簡単な言葉で答えようとします。

### 実装

RAG のアプローチに沿うために、このノートブックは LangChain フレームワークを使用しています。LangChain フレームワークは、RAG のようなパターンを効率的に構築するための様々なサービスやツールと統合されています。以下のツールを使います：

- **LLM (大規模言語モデル)**: Amazon Bedrock で使用可能な Anthropic Claude V1 

  このモデルは、ドキュメントのチャンクを理解し、人間が理解しやすい方法で答えを提供するために使用されます。
- **埋め込みモデル**: Amazon Bedrock で使用可能な Amazon Titan Embeddings

  このモデルは、テキスト文書の数値表現を生成するために使用されます。
- **ドキュメントローダー**: LangChain を通して利用可能な PDF Loader

  このノートブックでは、ローカルパスからサンプルファイルをロードするために、このドキュメントローダーを使用します。これは、企業の内部システムからドキュメントをロードするローダーに簡単に置き換えることができます。

- **ベクトルストア**: LangChain を通して利用可能な FAISS

  このノートブックでは、埋め込みとドキュメントの両方を保存するために、このインメモリのベクトルストアを使用しています。エンタープライズのユースケースでは、AWS OpenSearch、RDS for PostgreSQL with pgVector、ChromaDB、Pinecone、Weaviate のような永続的なストアに置き換えることができます。
- **インデックス**: VectorIndex

  このインデックスは、関連するドキュメントを見つけるために、入力の埋め込みとドキュメントの埋め込みを比較する場合に役立ちます。
- **ラッパー**: インデックス、ベクトルストア、埋め込みモデル、LLM をラップし、ロジックを抽象化します。

## セットアップ

このノートブックの残りの部分を実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrock に接続する必要があります。

セットアップの仕組みと ⚠️ **whether you might need to make any changes** についての詳細は、[Bedrock boto3 setup ノートブック](../00_Intro/bedrock_boto3_setup.ja.ipynb)  を参照してください。

このノートブックでは、追加の依存関係が必要になります：

- [FAISS](https://github.com/facebookresearch/faiss) ... 埋め込みベクトルを保存するため
- [PyPDF](https://pypi.org/project/pypdf/) ... PDF ファイルを操作するため

In [ ]:
# 事前にリポジトリルートから `download-dependencies.sh` を実行済みであることを確認してください!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet "faiss-cpu>=1.7,<2" langchain==0.0.249 "pypdf>=3.8,<4"

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ 必要に応じて AWS 設定に関する以下のコードのコメントを解除、編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## LangChain の設定


まずは LLM と埋め込み (Embedding) モデルのインスタンス化から始めます。ここでは、テキスト生成に Anthropic Claude を、テキスト埋め込みに Amazon Titan を使っています。

注意: Bedrock で用いることができる他のモデルを選択することも可能です。 `model_id` を以下のように置き換えることでモデルを変更可能です。

`llm = Bedrock(model_id="amazon.titan-tg1-large")`

使用可能な model ID:

- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v2`

テキスト埋め込みについても同様です:

`bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-g1-text-02")`

In [ ]:
# 埋め込みの生成には、 Titan Embeddings モデルを使用します。
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# Anthropic のモデルを作成します
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':200})
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-g1-text-02", client=boto3_bedrock)

## データ準備
はじめに、ドキュメントストアを作成するためにいくつかのファイルをダウンロードしましょう。今回の例では公開されている IRS のドキュメントを[こちら](https://www.irs.gov/publications)から使用します。

In [ ]:
from urllib.request import urlretrieve

os.makedirs("data", exist_ok=True)
files = [
    "https://www.irs.gov/pub/irs-pdf/p1544.pdf",
    "https://www.irs.gov/pub/irs-pdf/p15.pdf",
    "https://www.irs.gov/pub/irs-pdf/p1212.pdf",
]
for url in files:
    file_path = os.path.join("data", url.rpartition("/")[2])
    urlretrieve(url, file_path)

ダウンロードののち、[DirectoryLoader from PyPDF available under LangChain](https://python.langchain.com/en/latest/reference/modules/document_loaders.html)を参照しながらドキュメントをロードし、より小さいチャンクに分割することができます。

注意: 検索された文書/テキストは、質問に答えるのに十分な情報を含む大きさでなければなりませんが、さらに LLM プロンプトに収まる大きさでなければなりません。また、埋め込み (Embedding) モデルでは、入力トークンの長さを 8192 トークンまでに制限されており、これはおよそ 32,000 文字に相当します。このユースケースのために、私たちは[RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)を使って 100 文字のオーバーラップで約 1000 文字のチャンクを作成しています。

In [ ]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data/")

documents = loader.load()
# - 私たちのテストでは、文字分割はこの PDF データセットでより良く機能します。
text_splitter = RecursiveCharacterTextSplitter(
    # 表示するために、非常に小さいチャンクを指定してください。
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs = text_splitter.split_documents(documents)

In [ ]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)
avg_char_count_pre = avg_doc_length(documents)
avg_char_count_post = avg_doc_length(docs)
print(f'Average length among {len(documents)} documents loaded is {avg_char_count_pre} characters.')
print(f'After the split we have {len(docs)} documents more than the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_char_count_post} characters.')

3 つのPDFドキュメントがあり、それらは約 500 個の小さなチャンクに分割されています。

これで、わたしたちはこれらのチャンクについて、サンプルの埋め込みがどのように見えるのかを確認することができます。

In [ ]:
sample_embedding = np.array(bedrock_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

同様のパターンで、コーパス全体の埋め込みを生成し、ベクトルストアに保存することができます。

これは [LangChain](https://python.langchain.com/en/latest/modules/indexes/vectorstores/examples/faiss.html) 内の [FAISS](https://github.com/facebookresearch/faiss) 実装を使用して簡単に実行することができます。埋め込み (Embedding) モデルとドキュメントを入力してベクトルストア全体を作成します。
Index Wrapper を使うと、プロンプトの作成、クエリの埋め込みの取得、関連ドキュメントのサンプリング、LLM の呼び出しといった面倒な作業のほとんどを抽象化することができます。[VectorStoreIndexWrapper](https://python.langchain.com/en/latest/modules/indexes/getting_started.html#one-line-index-creation)  が有用です。

**⚠️⚠️⚠️ 注意: 次のセルの実行には数分かかる場合があります ⚠️⚠️⚠️**

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    bedrock_embeddings,
)

wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

## 質問応答

現在、ベクトルストアが用意できているため、質問を始めることができます。

In [ ]:
query = "Is it possible that I get sentenced to jail due to failure in filings?"

最初のステップは、ドキュメントと比較できるようにクエリの埋め込みを作成することです。

In [ ]:
query_embedding = vectorstore_faiss.embedding_function(query)
np.array(query_embedding)

このクエリの埋め込みを使用して、関連するドキュメントを取得できます。
クエリが埋め込みとして表現されるようになった今、クエリをデータストアに対して類似検索することで、最も関連性の高い情報が得られます。

In [ ]:
relevant_documents = vectorstore_faiss.similarity_search_by_vector(query_embedding)
print(f'{len(relevant_documents)} documents are fetched which are relevant to the query.')
print('----')
for i, rel_doc in enumerate(relevant_documents):
    print_ww(f'## Document {i+1}: {rel_doc.page_content}.......')
    print('---')

関連ドキュメントが揃ったので、今度は LLM を使用してこれらのドキュメントに基づいて回答を生成します。 

最初のプロンプトと、類似検索の結果に基づいて検索された関連ドキュメントを取得します。次に、これらを組み合わせてプロンプトを作成し、モデルにフィードバックして結果を取得します。

LangChain は、これらを容易に行うための抽象化を提供します。

### 簡単な方法
LangChain が提供するラッパーを使用することもできます。このラッパーはベクトルストアをラップし、LLM の入力を受け取ります。
このラッパーは、バックグラウンドで以下のステップを実行します。
- 質問を入力する
- 質問の Embedding を作成する
- 関連文書を取得する
- 書類と質問をプロンプトに詰め込む
- プロンプトを使用してモデルを起動し、人間の理解しやすい表現で回答を生成する

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """Human: {context}

Question: {question}

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

answer = wrapper_store_faiss.query(question=query, llm=llm, chain_type_kwargs={"prompt": PROMPT})
print_ww(answer)

違う質問をしてみましょう。

In [ ]:
query_2 = "What is the difference between market discount and qualified stated interest"

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = """Human: {context}

Question: {question}

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

answer_2 = wrapper_store_faiss.query(question=query_2, llm=llm, chain_type_kwargs={"prompt": PROMPT})
print_ww(answer_2)

### カスタマイズ可能なオプション

上記のシナリオでは、質問に対するコンテキストに応じた回答をすばやく簡単に取得する方法を検討しました。それでは、[RetrievalQA](https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html) を用いて、よりカスタマイズ可能なオプションを見てみましょう。これを用いると、`chain_type` パラメータを使用して、取得したドキュメントをプロンプトに追加する方法をカスタマイズできます。 また、取得する関連ドキュメントの数を制御したい場合は、下のセルの `k` パラメータを変更して異なる出力を確認してみてください。LLM が回答の生成に使用したソースドキュメントを知りたい場合も多いでしょう。その場合は、LLM プロンプトのコンテキストに追加されたドキュメントを返す `return_source_documents` を使用してそれらのドキュメントを Output として取得できます。`RetrievalQA` ではさらに、モデル固有にカスタムされた [プロンプトテンプレート](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/getting_started.html) を使用することもできます。

注意:この例では、Amazon Bedrock の LLM として Anthropic Claude を使用しています。この特定のモデルは、入力が `Human:` の後に提供され、モデルが `Assistant:` の後に出力を生成するように要求された場合に最も効果的です。[詳細はこちらをご参照ください](https://docs.anthropic.com/claude/docs/human-and-assistant-formatting) 下のセルでは、LLMがコンテキストを超えた回答をしないように、プロンプトを制御する方法の一例を見ることができます。

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

prompt_template = """

Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context

Question: {question}

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_faiss.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)
query = "Is it possible that I get sentenced to jail due to failure in filings?"
result = qa({"query": query})
print_ww(result['result'])

In [ ]:
result['source_documents']

## まとめ
あなたは検索拡張生成 (RAG) に関するモジュールを完了しました！これは、大規模言語モデルの能力と検索を組み合わせた重要な手法です。関連する検索例で生成を増やすことで、受け取った応答はより一貫性があり、かつ根拠のあるものになります。あなたはこの革新的なアプローチを学んだことを誇りに思うはずです。あなたが得た知識は、創造的で魅力的な言語生成システムを構築する際にかならず役立つでしょう。

上記の RAG ベースの質問応答機能の実装では、以下の概念と、Amazon Bedrock と LangChain の統合を使用してそれらを実装する方法を検討しました。

- ドキュメントのロードと埋め込みの生成によるベクトルストアの作成する
- 質問のための文書を取得する
- LLM への入力として送られるプロンプトの準備する
- 人間の理解しやすい表現で回答を提示する

### 発展要素
- さまざまなベクトルストアを試してみる
- Amazon Bedrock で入手可能なさまざまなモデルを活用して、出力を確認する
- 埋め込みやドキュメントチャンクの永続ストレージなどのオプションを検討する
- エンタープライズデータストアとの統合

# ありがとうございました!